In [1]:
import json

import gymnasium
import numpy as np
import torch


from lstm.lstmmodel import LSTM, LSTMDS, train_lstm
from vae.vaemodel import VAE
from vae.collect_data import collect_human_data, save_rollouts, load_rollouts

In [2]:
env = gymnasium.make("CarRacing-v2", render_mode="rgb_array", continuous=False)
rollouts = collect_human_data(env, keys_to_action={"w": 3, "a": 2, "d": 1, "s": 4}, continuous=True)

(1000, 96, 96, 3) (1000, 3) (1000, 1) (1000, 96, 96, 3)
(380, 96, 96, 3) (380, 3) (380, 1) (380, 96, 96, 3)


In [3]:
print(len(rollouts))
print(rollouts[0].obses.shape)

save_rollouts(rollouts, "lstm/human-data.npz")

rollouts = load_rollouts("lstm/human-data.npz")

3
(1000, 96, 96, 3)
Unpadding obses
Unpadding actions
Unpadding rewards
Unpadding next_obses
(1000, 96, 96, 3) (0, 3) (1000, 1) (1000, 96, 96, 3)
Unpadding obses
Unpadding actions
Unpadding rewards
Unpadding next_obses
(380, 96, 96, 3) (15, 3) (380, 1) (380, 96, 96, 3)
Unpadding obses
Unpadding actions
Unpadding rewards
Unpadding next_obses
(68, 96, 96, 3) (9, 3) (68, 1) (68, 96, 96, 3)
3
100


IndexError: index 994 is out of bounds for axis 0 with size 380

: 

In [ ]:


# Train VAE
# vae_ds = VAEDS(data)
with open("config.json", "r") as f:
    config = json.load(f)
vae = VAE(config["img_size"], config["latent_dim"], config["encoder_blocks"], config["decoder_blocks"])
# train_model(vae, vae_ds, 10, 64)
# torch.save(vae.state_dict(), "lstm/vae.pt")
vae.load_state_dict(torch.load("lstm/vae.pt", weights_only=True))

lstm_ds = LSTMDS(data, vae)
lstm = LSTM(32, 1, 64)
train_lstm(lstm, lstm_ds, 100, 64, log_dir="runs/exp3", device="mps")
torch.save(lstm.state_dict(), "lstm/lstm.pt")